## Moat Brand Reporting Refresh

In [26]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
%autoreload 2

In [37]:
from rtf_utils.moat_utils import MoatTile
from essence.analytics.platform import securedcredentials as secure_creds
from rtf_utils.gcp_utils import BigQuery,CloudStorage
import warnings
from rtf_utils.bq_schema import moat_schema_dict
import json
import logging

In [44]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
warnings.filterwarnings("ignore", "Your application has authenticated using end user credentials")

In [8]:
service_account_email='131786951123-compute@developer.gserviceaccount.com' ## gcp analytics service account
PROJECT_ID='essence-analytics-dwh'

credentialsFromVault=secure_creds.getCredentialsFromEssenceVault(service_account_email)
print("Creds Loaded")
moat_token = secure_creds.getDataFromEssenceVault('Moat_Token_Google')
print("Token Loaded")

Creds Loaded
Token Loaded


In [9]:
start_date = '20191001'
end_date = '20191006'

In [1]:
with open('dags/brand_report_task_queue.json') as json_file:
    config_json = json.load(json_file)
    campaigns = config_json['campaigns']

NameError: name 'json' is not defined

In [51]:
## Define Moat Tasks
Q4 = [(2506,{'level1':22783112},['date','level1','level3']), ## disp stadia,
      (2506,{'level1':23221764},['date','level1','level3']), ## disp nest 1,
      (2506,{'level1':23197607},['date','level1','level3']), ## disp nest 2
      (2506,{'level1':23278016},['date','level1','level3']), ## disp assistant
      (2698,{'level1':22783112},['date','level1','level3']), ## vid stadia,
      (2698,{'level1':23221764},['date','level1','level3']), ## vid nest 1,
      (2698,{'level1':23197607},['date','level1','level3']), ## vid nest 2,
      (2698,{'level1':23278016},['date','level1','level3']),
      (6179366,{'level2':2604566125},['date','level4']),## yt reserve - stadia
      (6178833,{'level1':1533586},['date','level4']) ## yt res dbm  - assistant
 ]
 # need access to 6196284

In [42]:
def get_moat_data(tile_id,start_date, end_date,level_filters=None,dimensions=None,**kwargs):
    tile = MoatTile(tile_id,level_filters,dimensions)
    local_filename = tile.get_data(start_date,end_date,moat_token)
    return local_filename

def upload_local_file(local_filename,dest_bucket,folder=None):
    gcs = CloudStorage(credentialsFromVault)
    if folder:
        blob_name = folder + "/" + local_filename
    else:
        blob_name = local_filename
    blob = gcs.upload_blob(dest_bucket, blob_name,local_filename,mode='filename')
    return blob

# Prototype WorkFlow

In [52]:
import os
import time
os.environ['GOOGLE_CLOUD_PROJECT'] = 'essence-analytics-dwh'
#for x in bonito:
for i,x in enumerate(Q4):
    time.sleep(10)
    tile_id, level_filter, dimensions = x
    print(tile_id, level_filter, dimensions)
    filename = get_moat_data(tile_id,start_date,end_date,level_filter,dimensions)
    
    if filename:
        print("Stored at {}".format(filename))
    else:
        print("No Data")
        
        continue    
    
    try:
        file_uri = upload_local_file(filename,"rtf_staging",folder="brand_reporting")
        file_uri = "gs://rtf_staging/" + file_uri.name
    
        print("Load Schema")
        #schema_dict = moat_schema_dict[tile_id]

        dest_table = "003_Moat_{}_{}".format(tile_id,i)

        print("Try BQ Upload: {}".format(dest_table))

        bq = BigQuery(credentialsFromVault)
        resp = bq.load_from_gcs("rtf_br_reporting",
                         file_uri,
                        dest_table,
                        #schema=schema_dict['schema'],
                        
                        extension='json')
    except Exception as e:
        print(e)
        continue

INFO:root:API Request Time


2506 {'level1': 22783112} ['date', 'level1', 'level3']


INFO:root:Moat_2506_22783112.json Saved


Stored at Moat_2506_22783112.json


INFO:root:File uploaded as brand_reporting/Moat_2506_22783112.json.


...

6178833 {'level1': 1533586} ['date', 'level4']


INFO:root:Moat_6178833_1533586.json Saved


Stored at Moat_6178833_1533586.json


INFO:root:File uploaded as brand_reporting/Moat_6178833_1533586.json.


Load Schema
Try BQ Upload: 003_Moat_6178833_9


In [50]:
file_uri = "gs://rtf_staging/brand_reporting/Brand Report - Upload File 2019-10-07T2046.csv"

dest_table = "003_DataLab_DailyPerformance"

bq.load_from_gcs("rtf_br_reporting",
                         file_uri,
                        dest_table)
                        #schema=schema_dict['schema']